In [1]:
import pandas as pd

In [2]:
full_df = pd.read_csv('dataset/train.csv')
df = full_df.copy()

In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_train_df(dataframe):
    imputers = {
        'name_imputer': SimpleImputer(strategy='constant', fill_value='Unknown Unknown'),
        'num_imputer': SimpleImputer(strategy='median'),
        'cat_imputer': SimpleImputer(strategy='constant', fill_value='Unknown'),
    }

    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].fit_transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].fit_transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].fit_transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)


    # std_scalers = {}

    # num_cols = df.select_dtypes('float').columns

    # for num_col in num_cols:
    #     scaler = StandardScaler()
    #     df.loc[:, num_col] = scaler.fit_transform(df.loc[:, num_col])

    #     std_scalers = 

    # Encoding 
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
    
    one_hot_encoder = OneHotEncoder(drop='first', dtype=int)
    
    cat_data_encoded = one_hot_encoder.fit_transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)
    
    return df, imputers, one_hot_encoder

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_test_df(dataframe, imputers, one_hot_encoder):
    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)

    # Encoding
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
        
    cat_data_encoded = one_hot_encoder.transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    return df

In [6]:
df_train_updated, train_imputers, train_ohe = update_train_df(df_train) 

/tmp/ipykernel_158699/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 1 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [7]:
df_test_updated = update_test_df(df_test, train_imputers, train_ohe)

In [8]:
X_train = df_train_updated.drop('Transported', axis=1)
y_train = df_train_updated.loc[:, 'Transported']

X_test = df_test_updated.drop('Transported', axis=1)
y_test = df_test_updated.loc[:, 'Transported']

In [9]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def objective(trial):
    params = {
        'C': trial.suggest_float('C', 1e-3, 100.0, log=True),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet']),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'saga']),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0) if 'elasticnet' in trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet']) else None,
        'max_iter': 1000
    }

    # Ensure compatibility between solver and penalty
    if params['penalty'] == 'l1' and params['solver'] == 'saga':
        pass  # valid
    elif params['penalty'] == 'l1' and params['solver'] != 'liblinear':
        return 0.0  # invalid combo
    elif params['penalty'] == 'elasticnet' and params['solver'] != 'saga':
        return 0.0  # invalid combo

    # Build pipeline with scaling
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(**params, random_state=42))
    ])

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best trial:")
print(study.best_trial.params)

[I 2025-07-07 23:57:06,401] A new study created in memory with name: no-name-9a18f5ad-2e70-481e-8455-502cec3fad17
[I 2025-07-07 23:57:06,483] Trial 0 finished with value: 0.7627288478347445 and parameters: {'C': 0.004333854083716797, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 0 with value: 0.7627288478347445.
[I 2025-07-07 23:57:06,775] Trial 1 finished with value: 0.7881791992717833 and parameters: {'C': 69.7464092122572, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 1 with value: 0.7881791992717833.
[I 2025-07-07 23:57:06,775] Trial 2 finished with value: 0.0 and parameters: {'C': 0.07917335040064177, 'penalty': 'elasticnet', 'solver': 'liblinear', 'l1_ratio': 0.2481918245146666}. Best is trial 1 with value: 0.7881791992717833.
/home/khabith/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-07-07 23:57:09,631] Trial 3 finished with v

Best trial:
{'C': 0.04365892318987059, 'penalty': 'l1', 'solver': 'liblinear'}


In [10]:
study.best_value

0.7899049904576698

In [11]:
train, train_imputers, train_ohe = update_train_df(full_df)

/tmp/ipykernel_158699/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [12]:
test_df = pd.read_csv('dataset/test.csv')
test = update_test_df(test_df, train_imputers, train_ohe)

In [13]:
X = train.drop('Transported', axis=1)
y = train.loc[:, 'Transported']

In [14]:
best_params = study.best_trial.params

In [15]:
best_model = model = Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(**best_params, random_state=42))
    ])
best_model.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 LogisticRegression(C=0.04365892318987059, penalty='l1',
                                    random_state=42, solver='liblinear'))])

In [16]:
y_pred = best_model.predict(test)
y_pre = y_pred.astype(bool)

In [17]:
test_df = pd.read_csv('dataset/test.csv')


submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported' : y_pred.astype(bool)
})
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [20]:
submission.to_csv('LR-79.csv', index=False)

In [21]:
best_params

{'C': 0.04365892318987059, 'penalty': 'l1', 'solver': 'liblinear'}